In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from datetime import time
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
import math
import random


/home/matheuspds/miniconda3/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence
/home/matheuspds/miniconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [45]:
pd_train = pd.read_csv('../processed_train_volume2.csv')

In [46]:
pd_train.head()

,time,tollgate_id,direction,vehicle_model,has_etc,vehicle_type,weekday,weekend,hour,date,holiday,time_window
0,2016-09-19 23:09:25,2,0,1,0,0,1,0,23,2016-09-19,False,"[23:00:00,23:20:00)"
1,2016-09-19 23:11:53,2,0,1,0,0,1,0,23,2016-09-19,False,"[23:00:00,23:20:00)"
2,2016-09-19 23:13:54,2,0,1,0,0,1,0,23,2016-09-19,False,"[23:00:00,23:20:00)"
3,2016-09-19 23:17:48,1,0,1,1,0,1,0,23,2016-09-19,False,"[23:00:00,23:20:00)"
4,2016-09-19 23:16:07,2,0,1,0,0,1,0,23,2016-09-19,False,"[23:00:00,23:20:00)"


In [47]:
pd_train['time'] =  pd.to_datetime(pd_volume_train['time'] , format='%Y-%m-%d %H:%M:%S')

pd_train = pd_train.groupby([pd.Grouper(freq='20T', key='time'), 'tollgate_id', 'direction', 'time_window', 'date', 'hour']).size()\
       .reset_index().rename(columns = {0:'volume'})

In [48]:
pd_train['weekday'] = pd_train['time'].dt.dayofweek + 1

In [49]:
pd_train.head()

,time,tollgate_id,direction,time_window,date,hour,volume,weekday
0,2016-09-19,1,0,"[00:00:00,00:20:00)",2016-09-19,0,13,1
1,2016-09-19,1,1,"[00:00:00,00:20:00)",2016-09-19,0,140,1
2,2016-09-19,2,0,"[00:00:00,00:20:00)",2016-09-19,0,2,1
3,2016-09-19,3,0,"[00:00:00,00:20:00)",2016-09-19,0,17,1
4,2016-09-19,3,1,"[00:00:00,00:20:00)",2016-09-19,0,181,1


In [52]:
pd_train[(pd_train['date'] == "2016-09-19") 
         & (pd_train['tollgate_id'] == 1) 
         & (pd_train['direction'] == 0)
         & (pd_train['weekday'] == 1)]

,time,tollgate_id,direction,time_window,date,hour,volume,weekday
0,2016-09-19 00:00:00,1,0,"[00:00:00,00:20:00)",2016-09-19,0,13,1
5,2016-09-19 00:20:00,1,0,"[00:20:00,00:40:00)",2016-09-19,0,6,1
9,2016-09-19 00:40:00,1,0,"[00:40:00,01:00:00)",2016-09-19,0,9,1
13,2016-09-19 01:00:00,1,0,"[01:00:00,01:20:00)",2016-09-19,1,10,1
18,2016-09-19 01:20:00,1,0,"[01:20:00,01:40:00)",2016-09-19,1,14,1
22,2016-09-19 01:40:00,1,0,"[01:40:00,02:00:00)",2016-09-19,1,10,1
27,2016-09-19 02:00:00,1,0,"[02:00:00,02:20:00)",2016-09-19,2,7,1
31,2016-09-19 02:20:00,1,0,"[02:20:00,02:40:00)",2016-09-19,2,10,1
35,2016-09-19 02:40:00,1,0,"[02:40:00,03:00:00)",2016-09-19,2,6,1
38,2016-09-19 03:00:00,1,0,"[03:00:00,03:20:00)",2016-09-19,3,9,1


In [53]:
pd_train.groupby(['time_window', 'weekday', 'direction', 'tollgate_id'])["volume"].mean()

time_window          weekday  direction  tollgate_id
[00:00:00,00:20:00)  1        0          1               14.000000
                                         2                2.333333
                                         3               17.600000
                              1          1              102.000000
                                         3              127.600000
                     2        0          1               12.000000
                                         2                2.000000
                                         3               17.250000
                              1          1               98.500000
                                         3              127.500000
                     3        0          1               14.500000
                                         2                2.666667
                                         3               17.750000
                              1          1               97.250000
         

In [24]:
df_remove = pd_train.loc[(pd_train['date'] >= '2016-10-01') 
                         & (pd_train['date'] <= '2016-10-07') 
                        ]

pd_train = pd_train.drop(df_remove.index)

In [25]:
pd_train.head()

,time,tollgate_id,direction,time_window,date,hour,volume,weekday
0,2016-09-19,1,0,"[00:00:00,00:20:00)",2016-09-19,0,13,1
1,2016-09-19,1,1,"[00:00:00,00:20:00)",2016-09-19,0,140,1
2,2016-09-19,2,0,"[00:00:00,00:20:00)",2016-09-19,0,2,1
3,2016-09-19,3,0,"[00:00:00,00:20:00)",2016-09-19,0,17,1
4,2016-09-19,3,1,"[00:00:00,00:20:00)",2016-09-19,0,181,1


In [26]:
pd_train[(pd_train['date'] == '2016-10-01') & (pd_train['hour'] == 8)].head()

,time,tollgate_id,direction,time_window,date,hour,volume,weekday


In [29]:
pd_train.tail()

,time,tollgate_id,direction,time_window,date,hour,volume,weekday
10058,2016-10-17 23:40:00,1,0,"[23:40:00,00:00:00)",2016-10-17,23,2,1
10059,2016-10-17 23:40:00,1,1,"[23:40:00,00:00:00)",2016-10-17,23,10,1
10060,2016-10-17 23:40:00,2,0,"[23:40:00,00:00:00)",2016-10-17,23,18,1
10061,2016-10-17 23:40:00,3,0,"[23:40:00,00:00:00)",2016-10-17,23,15,1
10062,2016-10-17 23:40:00,3,1,"[23:40:00,00:00:00)",2016-10-17,23,22,1


In [27]:
freq = "20min"

In [30]:
# movimenta a time_window 0 5 10 15 minutos para os dados de treino
range_1 = pd.date_range("2016-09-19 00:00:00", "2016-10-17 00:00:00", freq=freq)
range_2 = pd.date_range("2016-09-19 00:05:00", "2016-10-17 00:00:00", freq=freq)
range_3 = pd.date_range("2016-09-19 00:10:00", "2016-10-17 00:00:00", freq=freq)
range_4 = pd.date_range("2016-09-19 00:15:00", "2016-10-17 00:00:00", freq=freq)

In [31]:
def run(df,rng):
    rng_length = len(rng)
    result_dfs = []
    for this_direction in range(2):
        for this_tollgate_id in range(1, 4):
            time_start_list = []
            volume_list = []
            direction_list = []
            tollgate_id_list = []

            this_df = df[(df.tollgate_id == this_tollgate_id) & (df.direction == this_direction)]
            if len(this_df) > 0:
                for ind in range(rng_length - 1):
                    this_df_time_window = this_df[(this_df.time >= rng[ind]) & (this_df.time < rng[ind + 1])]
                    volume_list.append(len(this_df_time_window))

                    time_start_list.append(rng[ind])

                result_df = pd.DataFrame({'time_start': time_start_list,
                                          'volume': volume_list,
                                          'direction': [this_direction] * (rng_length - 1),
                                          'tollgate_id': [this_tollgate_id] * (rng_length - 1),
                }
                )
                result_dfs.append(result_df)

    d = pd.concat(result_dfs)

    if type == 'test':
        d['hour'] = d['time_start'].apply(lambda x: x.hour)
        dd = d[d.hour.isin([6, 7, 15, 16])]
    return d

In [32]:
df_train_0 = run(pd_train,range_1)

In [33]:
df_train_0.head()

,time_start,volume,direction,tollgate_id
0,2016-09-19 00:00:00,1,0,1
1,2016-09-19 00:20:00,1,0,1
2,2016-09-19 00:40:00,1,0,1
3,2016-09-19 01:00:00,1,0,1
4,2016-09-19 01:20:00,1,0,1


In [4]:
number_list = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70]
print ("Original list : ",  number_list)

random.shuffle(number_list) #shuffle method
print ("List after first shuffle  : ",  number_list)

Original list :  [7, 14, 21, 28, 35, 42, 49, 56, 63, 70]
List after first shuffle  :  [7, 63, 49, 56, 70, 35, 21, 42, 28, 14]
